In [3]:
import librosa
import numpy as np
import matplotlib as plt
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import json
import os

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report,accuracy_score,precision_score,recall_score

In [4]:
DATA_ROOT = '../data/openmic-2018'

if not os.path.exists(DATA_ROOT):
    raise ValueError('Did you forget to set `DATA_ROOT`?')

In [5]:
OPENMIC = np.load(os.path.join(DATA_ROOT, 'openmic-2018.npz'), allow_pickle=True)

In [6]:
#we will overwrite 'X' later
X, Y_true, Y_mask, sample_key = OPENMIC['X'], OPENMIC['Y_true'], OPENMIC['Y_mask'], OPENMIC['sample_key']

In [7]:
features=pd.read_csv('features.csv')
features

,sample_key,zcr_mean,zcr_std,sc_mean,sc_std,sb_mean,sb_std,sr_mean,sr_std,mfcc1_mean,...,mfcc9_mean,mfcc9std,mfcc10_mean,mfcc10std,mfcc11_mean,mfcc11std,mfcc12_mean,mfcc12std,mfcc13_mean,mfcc13std
0,000046_3840,0.036367,0.021923,296.517136,133.109808,288.159666,267.139540,288.159666,267.139540,147.188060,...,166.07562,216.15845,158.768600,209.785170,151.586730,203.43971,144.200240,197.13676,136.577680,190.80775
1,000135_483840,0.052411,0.013491,478.192757,141.199973,409.844515,135.532246,409.844515,135.532246,1282.769400,...,765.03280,495.68510,590.328800,558.758240,425.313660,616.81950,274.802150,663.55350,142.181600,694.35410
2,000139_119040,0.081234,0.014925,604.505859,189.731508,719.222449,139.070343,719.222449,139.070343,292.690280,...,214.49968,316.38672,198.657150,314.673430,186.793670,313.87470,177.732700,313.87650,170.007640,313.99084
3,000141_153600,0.053718,0.013248,462.825122,153.244140,391.783679,84.894649,391.783679,84.894649,188.996630,...,146.75562,140.59268,130.843980,137.768260,116.143745,135.27222,102.521700,133.01720,89.947320,130.88220
4,000144_30720,0.082449,0.034076,601.706717,329.074617,766.763811,375.408621,766.763811,375.408621,196.932450,...,172.74915,268.45245,166.110350,266.885220,159.133510,265.19022,151.693700,263.76523,143.998100,262.70773
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,155294_184320,0.050435,0.009352,508.286842,94.249039,266.331118,63.090107,266.331118,63.090107,147.754410,...,75.58570,66.08795,52.333115,52.021423,29.923525,44.92963,8.635664,46.96052,-11.322803,55.88632
19996,155295_76800,0.009752,0.004809,61.216410,42.815086,101.288169,53.390921,101.288169,53.390921,631.029240,...,866.60800,485.64580,861.181340,485.591300,855.655100,485.54526,850.119600,485.50488,844.646670,485.46692
19997,155307_211200,0.065545,0.022368,450.432458,192.751011,665.939788,285.751704,665.939788,285.751704,100.515335,...,87.85434,141.25677,81.044754,138.202670,74.912180,134.90385,69.371315,131.49872,64.451385,128.12172
19998,155310_372480,0.042145,0.015084,332.139261,146.760990,409.563611,169.775745,409.563611,169.775745,144.219150,...,139.86327,216.60248,130.948000,216.237840,122.950190,216.05978,116.407776,216.06279,111.108376,216.22318


In [8]:
features.columns

Index(['sample_key', 'zcr_mean', 'zcr_std', 'sc_mean', 'sc_std', 'sb_mean',
       'sb_std', 'sr_mean', 'sr_std', 'mfcc1_mean', 'mfcc1std', 'mfcc2_mean',
       'mfcc2std', 'mfcc3_mean', 'mfcc3std', 'mfcc4_mean', 'mfcc4std',
       'mfcc5_mean', 'mfcc5std', 'mfcc6_mean', 'mfcc6std', 'mfcc7_mean',
       'mfcc7std', 'mfcc8_mean', 'mfcc8std', 'mfcc9_mean', 'mfcc9std',
       'mfcc10_mean', 'mfcc10std', 'mfcc11_mean', 'mfcc11std', 'mfcc12_mean',
       'mfcc12std', 'mfcc13_mean', 'mfcc13std'],
      dtype='object')

In [9]:
#The data
# TODO what is "the data"??????????????????????????????????????????????????????????
X=np.array(features.iloc[:,2:]) # <---- magic number!!!!!


In [10]:
split_train = pd.read_csv(
    os.path.join(DATA_ROOT, "partitions/split01_train.csv"), header=None,
).squeeze("columns")
split_test = pd.read_csv(
    os.path.join(DATA_ROOT, "partitions/split01_test.csv"), header=None,
).squeeze("columns")

split_test.head()

0      000178_3840
1     000308_61440
2    000312_184320
3    000319_145920
4    000321_218880
Name: 0, dtype: object

In [11]:
print('# Train: {},  # Test: {}'.format(len(split_train), len(split_test)))

# Train: 14915,  # Test: 5085


In [12]:
train_set = set(split_train)
test_set = set(split_test)

In [13]:
# These loops go through all sample keys, and save their row numbers
# to either idx_train or idx_test
#
# This will be useful in the next step for slicing the array data
idx_train, idx_test = [], []

for idx, n in enumerate(sample_key):
    if n in train_set:
        idx_train.append(idx)
    elif n in test_set:
        idx_test.append(idx)
    else:
        # This should never happen, but better safe than sorry.
        raise RuntimeError('Unknown sample key={}! Abort!'.format(sample_key[n]))
        
# Finally, cast the idx_* arrays to numpy structures
idx_train = np.asarray(idx_train)
idx_test = np.asarray(idx_test)

In [14]:
# Finally, we use the split indices to partition the features, labels, and masks
X_train = X[idx_train]
X_test = X[idx_test]

Y_true_train = Y_true[idx_train]
Y_true_test = Y_true[idx_test]

Y_mask_train = Y_mask[idx_train]
Y_mask_test = Y_mask[idx_test]

In [15]:
# Print out the sliced shapes as a sanity check
print(X_train.shape)
print(X_test.shape)

(14915, 33)
(5085, 33)


In [16]:
with open(os.path.join(DATA_ROOT, 'class-map.json'), 'r') as f:
    class_map = json.load(f)
class_map

{'accordion': 0,
 'banjo': 1,
 'bass': 2,
 'cello': 3,
 'clarinet': 4,
 'cymbals': 5,
 'drums': 6,
 'flute': 7,
 'guitar': 8,
 'mallet_percussion': 9,
 'mandolin': 10,
 'organ': 11,
 'piano': 12,
 'saxophone': 13,
 'synthesizer': 14,
 'trombone': 15,
 'trumpet': 16,
 'ukulele': 17,
 'violin': 18,
 'voice': 19}

# Model training

Since the OPENMIC dataset is an ongoing project, and still acquiring annotations, we found it a bit hard to mold the data. Out of convienence, we are using the approach in [modelling baseline](https://github.com/cosmir/openmic-2018/blob/master/examples/modeling-baseline.ipynb), as suggested by the authors of the dataset. 

First, we are replicating the method of [PMICSF](https://arxiv.org/abs/1912.02606), and so, only looking at six instrument:

```python
    ['flute', 'guitar', 'organ', 'piano', 'trumpet', 'voice']
```

In [17]:
import numpy as np

instruments = ['accordion', 'banjo', 'bass', 'cello', 'clarinet',
       'cymbals', 'drums', 'flute', 'guitar', 'mallet_percussion', 'mandolin',
       'organ', 'piano', 'saxophone', 'synthesizer', 'trombone', 'trumpet',
       'ukulele', 'violin', 'voice']

article_instruments = ['flute', 'guitar', 'organ', 'piano', 'trumpet', 'voice']

features = ['zcr_mean', 'zcr_std', 'sc_mean',
       'sc_std', 'sb_mean', 'sb_std', 'sr_mean', 'sr_std', 'mfcc1_mean',
       'mfcc1std', 'mfcc2_mean', 'mfcc2std', 'mfcc3_mean', 'mfcc3std',
       'mfcc4_mean', 'mfcc4std', 'mfcc5_mean', 'mfcc5std', 'mfcc6_mean',
       'mfcc6std', 'mfcc7_mean', 'mfcc7std', 'mfcc8_mean', 'mfcc8std',
       'mfcc9_mean', 'mfcc9std', 'mfcc10_mean', 'mfcc10std', 'mfcc11_mean',
       'mfcc11std', 'mfcc12_mean', 'mfcc12std', 'mfcc13_mean', 'mfcc13std']

article_features = ['zcr_mean', 'sc_mean', 'sb_mean', 'sr_mean', 'mfcc1_mean',
       'mfcc2_mean', 'mfcc3_mean', 'mfcc4_mean', 'mfcc5_mean', 'mfcc6_mean',
       'mfcc7_mean', 'mfcc8_mean', 'mfcc9_mean', 'mfcc10_mean', 'mfcc11_mean',
       'mfcc12_mean', 'mfcc13_mean']

We have the choice to replicate the article features and instruments, or picking some additional features we used.
Our baseline is the article features:

In [18]:
FEATURES = article_features
INSTRUMENTS = article_instruments

## Modeling baseline

We took the approach of previously mentioned [modelling baseline](https://github.com/cosmir/openmic-2018/blob/master/examples/modeling-baseline.ipynb) and turned it into a method:

In [19]:
def modelling_baseline(X_train, X_test, Y_true_train, Y_true_test, Y_mask_train, Y_mask_test, class_map, clf, instruments, output_dict=True):
    """
    Use the modeling baseline to train a classifier for each instrument
    Returns randomforrest_models - classifiers for trained model
            reports - classification reports for each instrument
            accuracies - accuracies for each instrument
    """
    # This dictionary will include the classifiers for each model
    models = dict()
    reports = dict()
    accuracies = dict()


    # We'll iterate over all istrument classes, and fit a model for each one
    # After training, we'll print a classification report for each instrument
    for instrument in instruments:

        # Map the instrument name to its column number
        inst_num = class_map[instrument]

        # Step 1: sub-sample the data

        # First, we need to select down to the data for which we have annotations
        # This is what the mask arrays are for
        train_inst = Y_mask_train[:, inst_num]
        test_inst = Y_mask_test[:, inst_num]

        # Here, we're using the Y_mask_train array to slice out only the training examples
        # for which we have annotations for the given class
        X_train_inst = X_train[train_inst]

        # Again, we slice the labels to the annotated examples
        # We thresold the label likelihoods at 0.5 to get binary labels
        Y_true_train_inst = Y_true_train[train_inst, inst_num] >= 0.5

        # Repeat the above slicing and dicing but for the test set
        X_test_inst = X_test[test_inst]
        Y_true_test_inst = Y_true_test[test_inst, inst_num] >= 0.5

        # Step 3.
        # Initialize a new classifier
        # No

        # Step 4.
        clf.fit(X_train_inst, Y_true_train_inst)

        # Step 5.
        # Finally, evaluate the model on both test data
        Y_pred_test = clf.predict(X_test_inst)
        
        # Store the classifier in our dictionary
        models[instrument] = clf
        reports[instrument] = classification_report(Y_true_test_inst, Y_pred_test, output_dict=output_dict)
        accuracies[instrument] = accuracy_score(Y_true_test_inst, Y_pred_test)

    return models, reports, accuracies
        

Let's store all results in a dictionary. If you want to find the result of a specific model, you can use the following baseline.

```python
    >>> results['RandomForest']['Accuracy']['flute']
    0.5925925925925926
```

In [20]:
keys = {"model": None, "report": None, "accuracy": None}

models = ['RandomForest', 'SVM', 'LogisticRegression', 'GradientBoost', 'DecisionTree', 'LGBM']

results = dict()

for model in models:
    results[model] = dict(keys)

"""results = {
    'RandomForest': dict(keys),
    'SVM': dict(keys),
    'LogisticRegression': dict(keys),
    'GradientBoost': dict(keys),
    'DecisionTree': dict(keys),
    'LGBM': dict(keys),
}"""

results

{'RandomForest': {'model': None, 'report': None, 'accuracy': None},
 'SVM': {'model': None, 'report': None, 'accuracy': None},
 'LogisticRegression': {'model': None, 'report': None, 'accuracy': None},
 'GradientBoost': {'model': None, 'report': None, 'accuracy': None},
 'DecisionTree': {'model': None, 'report': None, 'accuracy': None},
 'LGBM': {'model': None, 'report': None, 'accuracy': None}}

#  Random Forest Classifier

In [21]:
clf = RandomForestClassifier(n_estimators=100, random_state=42)

model, reports, accuracies = modelling_baseline(X_train, 
                                                X_test, 
                                                Y_true_train, 
                                                Y_true_test, 
                                                Y_mask_train, 
                                                Y_mask_test, 
                                                class_map, 
                                                clf, 
                                                INSTRUMENTS)

results['RandomForest']['model'] = model
results['RandomForest']['report'] = reports
results['RandomForest']['accuracy'] = accuracies

# SVM

In [22]:
# UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. 
# Use `zero_division` parameter to control this behavior.
clf = SVC()

model, reports, accuracies = modelling_baseline(X_train, 
                                                X_test, 
                                                Y_true_train, 
                                                Y_true_test, 
                                                Y_mask_train, 
                                                Y_mask_test, 
                                                class_map, 
                                                clf, 
                                                INSTRUMENTS)

results['SVM']['model'] = model
results['SVM']['report'] = reports
results['SVM']['accuracy'] = accuracies

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Decision Trees

In [23]:
clf = DecisionTreeClassifier()

model, reports, accuracies = modelling_baseline(X_train, 
                                                X_test, 
                                                Y_true_train, 
                                                Y_true_test, 
                                                Y_mask_train, 
                                                Y_mask_test, 
                                                class_map, 
                                                clf, 
                                                INSTRUMENTS)

results['DecisionTree']['model'] = model
results['DecisionTree']['report'] = reports
results['DecisionTree']['accuracy'] = accuracies

# Gradient Boosting

In [24]:
clf = GradientBoostingClassifier(learning_rate=0.3)

model, reports, accuracies = modelling_baseline(X_train, 
                                                X_test, 
                                                Y_true_train, 
                                                Y_true_test, 
                                                Y_mask_train, 
                                                Y_mask_test, 
                                                class_map, 
                                                clf, 
                                                INSTRUMENTS)

results['GradientBoost']['model'] = model
results['GradientBoost']['report'] = reports
results['GradientBoost']['accuracy'] = accuracies

# Logistic Regression

In [25]:
clf = LogisticRegression(max_iter=50)

model, reports, accuracies = modelling_baseline(X_train, 
                                                X_test, 
                                                Y_true_train, 
                                                Y_true_test, 
                                                Y_mask_train, 
                                                Y_mask_test, 
                                                class_map, 
                                                clf, 
                                                INSTRUMENTS)

results['LogisticRegression']['model'] = model
results['LogisticRegression']['report'] = reports
results['LogisticRegression']['accuracy'] = accuracies

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

# LGBM

In [26]:
clf = LGBMClassifier()

model, reports, accuracies = modelling_baseline(X_train, 
                                                X_test, 
                                                Y_true_train, 
                                                Y_true_test, 
                                                Y_mask_train, 
                                                Y_mask_test, 
                                                class_map, 
                                                clf, 
                                                INSTRUMENTS)

results['LGBM']['model'] = model
results['LGBM']['report'] = reports
results['LGBM']['accuracy'] = accuracies

# Results

In [27]:
m = np.mean(list(results['SVM']['accuracy'].values()))
std = np.std(list(results['SVM']['accuracy'].values()))
print(m, std)

0.727461493998946 0.06975715340387725


In [74]:
# den här outputten kan man använda för att skapa ett table med copilot
#instrument = 'flute'
#model = 'DecisionTree'
article_columns = ['P', 'R', 'F1', 'support']

columns = ["precision",	"recall", "f1-score", "support", "model", "instrument"]
df = pd.DataFrame(columns=columns)
df = df.rename(columns={ "precision": "P", 
                    "recall": "R", 
                    "f1-score": "F1"
                    }, inplace=True)

for instrument in INSTRUMENTS:
    for model in models:
        di = results[model]["report"][instrument]['True']
        di['model'] = model
        di['instrument'] = instrument
        df = pd.concat([df, pd.DataFrame(di, index=[0]).rename(columns={ "precision": "P", 
                    "recall": "R", 
                    "f1-score": "F1",
                    }).drop(columns=['support'])], ignore_index=True)
        #df.to_latex(buf="../tables/" + model + ".tex", index=False)

n_rows, n_cols = df.shape
df_melted = df.melt(id_vars=["model", "instrument"], var_name="metric", value_name="value")

df_pivot = df_melted.pivot_table(columns=['model', "metric"], 
                        index=['instrument'], 
                        values=['value'], 
                        aggfunc='mean'
                        )
#df_tex = df_pivot.style.to_latex(float_format="%.2f", 
#                            multicolumn=True, 
#                            multirow=True)
#df_tex = df_tex.replace("multirow{6}{*}{", "multirow{6}{*}{\\rotatebox[origin=c]{90}{")

df_pivot


value                                                        \
model      DecisionTree                     GradientBoost                       
metric               F1         P         R            F1         P         R   
instrument                                                                      
flute          0.425197  0.393204  0.462857      0.342657  0.441441  0.280000   
guitar         0.770035  0.767361  0.772727      0.818923  0.767584  0.877622   
organ          0.381743  0.383333  0.380165      0.364486  0.419355  0.322314   
piano          0.857658  0.881481  0.835088      0.902439  0.896194  0.908772   
trumpet        0.492308  0.481928  0.503145      0.499145  0.546816  0.459119   
voice          0.706131  0.670683  0.745536      0.810484  0.738971  0.897321   

                                                                       \
model           LGBM                     LogisticRegression             
metric            F1         P         R                 F1         P   
instrument                                                              
flute       0.377224  0.500000  0.302857           0.081218  0.363636   
guitar      0.819835  0.777429  0.867133           0.812983  0.728532   
organ       0.339623  0.395604  0.297521           0.233129  0.452381   
piano       0.901893  0.885135  0.919298           0.900169  0.869281   
trumpet     0.519737  0.544828  0.496855           0.369295  0.542683   
voice       0.808765  0.730216  0.906250           0.797665  0.706897   

                                                                           \
model                RandomForest                           SVM             
metric             R           F1         P         R        F1         P   
instrument                                                                  
flute       0.045714     0.328000  0.546667  0.234286  0.000000  0.000000   
guitar      0.919580     0.831424  0.781538  0.888112  0.820059  0.709184   
organ       0.157025     0.366492  0.500000  0.289256  0.211921  0.533333   
piano       0.933333     0.902998  0.907801  0.898246  0.905405  0.872964   
trumpet     0.279874     0.474048  0.526923  0.430818  0.473394  0.568282   
voice       0.915179     0.792829  0.715827  0.888393  0.803150  0.718310   

                      
model                 
metric             R  
instrument            
flute       0.000000  
guitar      0.972028  
organ       0.132231  
piano       0.940351  
trumpet     0.405660  
voice       0.910714

In [76]:
df_pivot[df_pivot.columns[0:9]].style.format("{:.2f}").background_gradient(cmap='OrRd')


In [77]:
df_pivot[df_pivot.columns[9:18]].style.format("{:.2f}").background_gradient(cmap='OrRd') #[['F1', 'P', 'R']]

In [1]:
#df_tex = df.groupby(['model', 'instrument']).mean().to_latex(float_format="%.2f", multicolumn=True, multirow=True)

NameError: name 'df' is not defined

$$\\begin{array}{rrr}\n\\toprule\n precision &   recall &  f1-score \\\\\n  0.386935 & 0.440000 &  0.411765 \\\\\n\\midrule\n  0.755172 & 0.765734 &  0.760417 \\\\\n  0.373984 & 0.380165 &  0.377049 \\\\\n  0.876866 & 0.824561 &  0.849910 \\\\\n  0.501475 & 0.534591 &  0.517504 \\\\\n  0.673387 & 0.745536 &  0.707627 \\\\\n  0.442478 & 0.285714 &  0.347222 \\\\\n  0.772308 & 0.877622 &  0.821604 \\\\\n  0.463415 & 0.314050 &  0.374384 \\\\\n  0.895833 & 0.905263 &  0.900524 \\\\\n  0.548507 & 0.462264 &  0.501706 \\\\\n  0.742537 & 0.888393 &  0.808943 \\\\\n  0.500000 & 0.302857 &  0.377224 \\\\\n  0.777429 & 0.867133 &  0.819835 \\\\\n  0.395604 & 0.297521 &  0.339623 \\\\\n  0.885135 & 0.919298 &  0.901893 \\\\\n  0.544828 & 0.496855 &  0.519737 \\\\\n  0.730216 & 0.906250 &  0.808765 \\\\\n  0.363636 & 0.045714 &  0.081218 \\\\\n  0.728532 & 0.919580 &  0.812983 \\\\\n  0.452381 & 0.157025 &  0.233129 \\\\\n  0.869281 & 0.933333 &  0.900169 \\\\\n  0.542683 & 0.279874 &  0.369295 \\\\\n  0.706897 & 0.915179 &  0.797665 \\\\\n  0.546667 & 0.234286 &  0.328000 \\\\\n  0.781538 & 0.888112 &  0.831424 \\\\\n  0.500000 & 0.289256 &  0.366492 \\\\\n  0.907801 & 0.898246 &  0.902998 \\\\\n  0.526923 & 0.430818 &  0.474048 \\\\\n  0.715827 & 0.888393 &  0.792829 \\\\\n  0.000000 & 0.000000 &  0.000000 \\\\\n  0.709184 & 0.972028 &  0.820059 \\\\\n  0.533333 & 0.132231 &  0.211921 \\\\\n  0.872964 & 0.940351 &  0.905405 \\\\\n  0.568282 & 0.405660 &  0.473394 \\\\\n  0.718310 & 0.910714 &  0.803150 \\\\\n\\bottomrule\n\\end{array}\n$$

In [113]:
import pandas as pd

df = pd.DataFrame(results)

for model in models:
    for instrument in INSTRUMENTS:
        print("#" * 52)
        print("{} - {}, Accuracy: {}\n".format(model, instrument, results[model]['accuracy'][instrument]))
        print(results[model]['report'][instrument])
    
    break

df.head()

,RandomForest,SVM,LogisticRegression,GradientBoost,DecisionTree,LGBM
model,{'flute': (DecisionTreeClassifier(max_features...,"{'flute': SVC(), 'guitar': SVC(), 'organ': SVC...","{'flute': LogisticRegression(max_iter=50), 'gu...",{'flute': ([DecisionTreeRegressor(criterion='f...,"{'flute': DecisionTreeClassifier(), 'guitar': ...","{'flute': LGBMClassifier(), 'guitar': LGBMClas..."
report,{'flute': ' precision recall ...,{'flute': ' precision recall ...,{'flute': ' precision recall ...,{'flute': ' precision recall ...,{'flute': ' precision recall ...,{'flute': ' precision recall ...
accuracy,"{'flute': 0.701067615658363, 'guitar': 0.76376...","{'flute': 0.6886120996441281, 'guitar': 0.7201...","{'flute': 0.6779359430604982, 'guitar': 0.7224...","{'flute': 0.6690391459074733, 'guitar': 0.7477...","{'flute': 0.6120996441281139, 'guitar': 0.6834...","{'flute': 0.6886120996441281, 'guitar': 0.75, ..."
